<a href="https://colab.research.google.com/github/gbardaklis/Mini-Project-1/blob/main/Q3Embeddings_as_Features_Wiki_News_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Needed Imports
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# More imports
import json
import pandas as pd
import numpy as np
import warnings

In [5]:
with open("/content/goemotions.json") as f:
    # data2 = json.loads(f.read())
    data = json.load(f)

# importing the json file into a dataframe 
df = pd.DataFrame(data, columns = ['Post', 'Emotion', 'Sentiment'])
postsDict = df['Post'].values.tolist()
# postsDict

In [6]:
emotionDict = df['Emotion'].values.tolist()
# emotionDict
sentimentDict = df['Sentiment'].values.tolist()
# sentimentDict

In [7]:
# Displays a List of Pretrained Models avail on gensim
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [8]:
# 3.1 
corpusNews = api.load('fasttext-wiki-news-subwords-300')

[=================================================-] 98.5% 944.4/958.4MB downloaded

In [9]:
# 3.2
tokenCount = 0
tokenPost = []

for x in postsDict:
  tokens = word_tokenize(x)
  tokenPost.append(tokens)
  tokenCount = tokenCount + len(tokens)
print(f'The total number of tokens is {tokenCount}')

The total number of tokens is 2642128


In [10]:
# fail counter for punctuation and words not found in corpus 
failed = 0
averageEmb = []
averageEmbAll = []

# Take the X TokenPost
for index, iterator in enumerate(tokenPost):
  for x in iterator:
    try:
      # Assign Corpus[x] to single word
      singleWord = corpusNews[x]
      # New List append that word
      averageEmb.append(singleWord)
    except KeyError:
      # Doesn't fnd throw error increment counter
      failed += 1
  if len(averageEmb) != 0:
    average = np.average(averageEmb, axis = 0)
    averageEmbAll.append(average)
    averageEmb.clear()
  else: 
    # Set the vector to 0 
    averageEmbAll.append([0] * 300)
# 3.3
print(f'The failed count is: {failed}')

# 3.4 
hitRate = ((tokenCount - failed) / tokenCount) *100
print(f'The hit rate is as follows: {hitRate}')

The failed count is: 51843
The hit rate is as follows: 98.03783162662823


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# splitting data 80% train 20% test
post_train, post_test, emotion_train, emotion_test = train_test_split(averageEmbAll, emotionDict, test_size = 0.2)

In [ ]:
# splitting data 80% train 20% test
post_train_sent, post_test_sent, sentiment_train, sentiment_test = train_test_split(averageEmbAll, sentimentDict, test_size = 0.2)

In [ ]:
# 3.5 Base-MLP with the default parameters
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(early_stopping=True)
%time mlp.fit(post_train, emotion_train)
%time mlp.fit(post_train, sentiment_train)

CPU times: user 14min 9s, sys: 5min 29s, total: 19min 39s
Wall time: 10min 15s
CPU times: user 4min 32s, sys: 2min 5s, total: 6min 37s
Wall time: 3min 28s


MLPClassifier(early_stopping=True)

In [ ]:
# 3.7
# Open Text Word doc for Writing to
performanceTxt2 = open("performanceWordEmb_Wiki_News2.txt", "w")
# Write to Text Word Doc the Accuracy and the Classification Report for BASE 
performanceTxt2.write(f" The following is the information for the Base MLP Classifier \n  " )
performanceTxt2.write(f" This is the Accuracy of the Tests for the emotion model \n {mlp.score(post_test, emotion_test) * 100} \n  " )
performanceTxt2.write(f" This is the Accuracy of the Tests for the sentiment model \n {mlp.score(post_test, sentiment_test) * 100} \n  " )
emotion_pred = mlp.fit(post_train, emotion_train).predict(post_test)
performanceTxt2.write(f" This is the Classification Report for the emotion model \n {metrics.classification_report(emotion_test, emotion_pred,labels=np.unique(emotion_test), zero_division=0)} \n  " )
sentiment_pred = mlp.fit(post_train_sent, sentiment_train).predict(post_test_sent)
performanceTxt2.write(f" This is the Classification Report for the sentiment model \n {metrics.classification_report(sentiment_test, sentiment_pred,labels=np.unique(sentiment_test), zero_division=0)} \n  " )
print("Wrote everything to the file good job!")

Wrote everything to the file good job!
